In [1]:
import time

# Save timestamp
start = time.time()




import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt
import json
from datetime import datetime

In [2]:
print(tf.version.VERSION)

2.18.0-dev20240611


### Specify here the NN topology and the dataframe to be loaded

In [3]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 32
hidden_layer_nodes = 31
output_layer_nodes = 3


use_case = "os-detection"
dataset_folder = f"../datasets/nprint-raw/{use_case}/"
features_rankings = f"../datasets/nprint-raw/{use_case}/feature-importance.csv"



###Define the model

In [4]:
BATCH_SIZE = 512
EPOCHS = 500

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [5]:
df_train = pd.read_csv(f'{dataset_folder}X.csv')
df_test = pd.read_csv(f'{dataset_folder}X_val.csv')
label_train = pd.read_csv(f'{dataset_folder}y.csv')
label_test = pd.read_csv(f'{dataset_folder}y_val.csv')

# df_train.drop('pcap', axis=1, inplace=True)
# df_test.drop('pcap', axis=1, inplace=True)
# label_train.drop('pcap', axis=1, inplace=True)
# label_test.drop('pcap', axis=1, inplace=True)
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)

# display(df_train)
# display(df_test)
# display(label_train)
display(label_test)

,pcap,label
0,485057076225527219_ubuntu_web-server,1
1,1152879459140876005_windows_windows-8.1,2
2,1229535654243745902_ubuntu_ubuntu-16.4-64b,1
3,584301198936813692_ubuntu_ubuntu-16.4-64b,1
4,203665707225169563_windows_windows-7-pro,2
...,...,...
739,1186303398913338167_ubuntu_ubuntu-16.4-64b,1
740,934697405162209634_windows_windows-10-pro,2
741,1195865193455630139_ubuntu_ubuntu-14.4-32b,1
742,749383642870436473_windows_windows-10-pro,2


In [6]:
#select a sub-dataframe with only the top attrs rate by the AUTOGLUON feature importance algorithm
#load the file with the rankings
features_rankings_df = pd.read_csv(features_rankings)
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

#make a list with only the best features
feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:number_of_attributes]
# print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]
# display(df_train)
# display(df_test)

In [7]:
#ONLY FOR VIDEO STREAM USE CASE
#RTS values conversion for video-streaming use-case
# INTEGER = 8

# mini = []
# maxi = []
# column_list = ["pkt_1_rts", "pkt_2_rts", "pkt_3_rts", "pkt_4_rts", "pkt_5_rts", "pkt_6_rts", "pkt_7_rts", "pkt_8_rts"]
# for column_name in column_list:
#     mini.append(df_test[column_name].min())
#     print(mini)    
#     maxi.append(df_test[column_name].max())
#     print(maxi)

# maximo = max(maxi)
# minimo = min(mini)
# print(maximo)


# display(df_test)
# for column_name in column_list:
#     aux = []
#     for x in df_test[column_name]:
#         normalized_value = (x - minimo) / (maximo - minimo)
#         scaled_value = normalized_value * (2**INTEGER - 1)
#         aux.append(scaled_value)
#     df_test[column_name]=aux    
# display(df_test)

# mini = []
# maxi = []
# for column_name in column_list:
#     mini.append(df_train[column_name].min())
#     print(mini)    
#     maxi.append(df_train[column_name].max())
#     print(maxi)

# maximo = max(maxi)
# minimo = min(mini)
# print(maximo)


# display(df_train)
# for column_name in column_list:
#     aux = []
#     for x in df_train[column_name]:
#         normalized_value = (x - minimo) / (maximo - minimo)
#         scaled_value = normalized_value * (2**INTEGER - 1)
#         aux.append(scaled_value)
#     df_train[column_name]=aux
    
# display(df_train)

### NN training

In [8]:
# Begin NN training
NUMBER_OF_ATTRS = df_train.shape[1]

# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

# Execute Trainning 
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, label_train, epochs=EPOCHS, verbose=2, batch_size=BATCH_SIZE)

# and Testing
test_loss, one_test_acc =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
tf_predictions_probabilities = model.predict(df_test)

#Crate Confusion Matrix for better understanding of results
tf_predictions = []
for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
print(conf_m)

Starting training for 32 attributes


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# Model info
model.summary()

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

In [ ]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS
    #"number_of_samples": number_of_samples,
    #"df_number": df_number
}

In [ ]:
#TO DO: use save_models instead of save_weights

# Model parameters json file generation, create file with date-time string to prevent unwated/accidental overwrites
os.chdir('../tf-params-reports/')
print(os.getcwd())

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

# Model file with weights and other params
os.chdir('../tf-models/') 
print(os.getcwd())

title_model_save = f'nn-nprint-{use_case}-model-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.keras'
model.save(title_model_save)

# Model weights file generation
# os.chdir('../tf-model-weights/')
# print(os.getcwd())

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.
    
# model.load_weights(f'nn-nprint-app-iden-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5')

In [ ]:
# Save timestamp
end = time.time()
print(end - start)

In [ ]:

# Acervo - comandos úteis os python
# %pwd
# os.chdir(C:\NN-P4\nn-reports)
# cwd = os.getcwd()
# print(cwd)
# os.listdir()
# os.chdir('../nn-reports/')

In [ ]:
# df_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X.pkl')
# df_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X_val.pkl')
# label_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y.pkl')
# label_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y_val.pkl')